# **Prueba Shor Adiabatico**

In [1]:
import dimod
import numpy as np
from dwave.system import LeapHybridCQMSampler
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
from time import time

En este cuadernillo se implementará el algoritmo de Shor según los argumentos expuestos en la sección 2.1 del informe. En primer lugar, trataremos de formular el CQM correspondiente para ejecutarlo en _D-Wave_. Posteriormente se realizará la digitalización en _Qiskit_.

## CQM en _D-Wave_

Comenzamos definiendo $N$ y calculando el número de _qubits_ necesario para el modelo. Se toma $N = 85079$.

In [2]:
N = 571*149   # N = xy, con x <= y

# Cálculo de qubits
if np.floor(np.sqrt(N))%2:                                  # Parte entera impar
    nx = int(np.ceil(np.log2(np.floor(np.sqrt(N)))) - 1)
else:
    nx = int(np.ceil(np.log2(np.floor(np.sqrt(N)-1))) - 1)

ny = int(np.ceil(np.log2(np.floor(N/3))) - 1)

# qubits totales
total_qubits = nx+ny
print(f"Son necesarios {total_qubits} qubits.\nn_x = {nx}.\nn_y = {ny}.")

Son necesarios 22 qubits.
n_x = 8.
n_y = 14.


Creamos las variables del problema de optimización: $x$ e $y$.

In [3]:
# Variables del problema
variables = [dimod.Integer("x"),dimod.Integer("y")]
print(variables)

[QuadraticModel({'x': 1.0}, {}, 0.0, {'x': 'INTEGER'}, dtype='float64'), QuadraticModel({'y': 1.0}, {}, 0.0, {'y': 'INTEGER'}, dtype='float64')]


Ya hemos tenido en cuenta la naturaleza entera de las variables. Establecemos a continuación los rangos de cada una de ellas.

In [4]:
# Límites para x
variables[0].set_upper_bound("x",(np.sqrt(N)-1)/2)  # Superior
variables[0].set_lower_bound("x",1)                 # Inferior

# Límites para y
variables[1].set_upper_bound("y",(N/3-1)/2)         # Superior
variables[1].set_lower_bound("y",(np.sqrt(N)-1)/2)  # Inferior

Habiendo hecho esto, podemos empezar a crear el CQM.

In [5]:
# Creamos el modelo CQM
cqm = dimod.ConstrainedQuadraticModel()

# Función objetivo
cqm.set_objective(N - (2*variables[0]+1)*(2*variables[1]+1))

# Restricción para la transformación en QUBO
cqm.add_constraint(N - (2*variables[0]+1)*(2*variables[1]+1)>=0)

# Mostramos el modelo
print(cqm)

Constrained quadratic model: 2 variables, 1 constraints, 6 biases

Objective
  85078 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y')

Constraints
  cbed21b: 85078 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y') >= 0.0

Bounds
  1.0 <= Integer('x') <= 145.3415235795348
  145.3415235795348 <= Integer('y') <= 14179.333333333334



Una vez definido, solo nos queda ejecutarlo en _D-Wave_ con la ayuda de un _sampler_ híbrido.

In [6]:
# Definimos el sampler híbrido
sampler = LeapHybridCQMSampler()

# Resolvemos el CQM
sampleset = sampler.sample_cqm(cqm)

# Tomamos las soluciones factibles
t0 = time()
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
t1 = time()
print(f"Obtenido mínimo en {t1-t0} segundos.")

# Mostramos la mejor solución. Recordemos que en caso de haber factorizado N = p*q serán p = 2*x_bar + 1 = x y q = 2*y_bar + 1 = y.
best = feasible_sampleset.first.sample
x_bar, y_bar = best["x"], best["y"]
x = int(2*x_bar+1)
y = int(2*y_bar+1)
print(f"x_bar = {x_bar}. x = {x}.")
print(f"y_bar = {y_bar}. y = {y}.")
if N == x*y:
    print(f"\nFactorización correcta. {N} = {x}*{y}.")
else:
    print("\nFactorización incorrecta.")

Obtenido mínimo en 8.170473098754883 segundos.
x_bar = 74.0. x = 149.
y_bar = 285.0. y = 571.

Factorización correcta. 85079 = 149*571.


Vista la implementación adiabática, pasemos ahora a DAdQC.

## Digitalización en _Qiskit_